In [1]:
import pandas as pd

df_train = pd.read_csv(r"C:\Users\trent\github_repos\titanic-0\train.csv")
df_test = pd.read_csv(r"C:\Users\trent\github_repos\titanic-0\test.csv")

df_train.set_index('PassengerId',inplace=True)
df_test.set_index('PassengerId',inplace=True)


df_train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
from myFunctions import MissingData
import myFunctions as myFn

df = df_train.copy()


categoricalNumericFeatures = ['Pclass','Survived'] # pclass is numeric but actually is a categorical feature
missingData = MissingData(df, categoricalImputer='most_frequent',numericImputer='mean', categoricalNumericFeatures=categoricalNumericFeatures, dropColPct=.7)
missingData.missing_values_report()

Survived	0.0%
Pclass	0.0%
Name	0.0%
Sex	0.0%
Age	19.87%
SibSp	0.0%
Parch	0.0%
Ticket	0.0%
Fare	0.0%
Cabin	77.1%
Embarked	0.22%


In [3]:
df = missingData.fillMissingData()

dropped the Cabin column. Over 70% of records were Null.
imputed 2 categorical column(s) using most_frequent.
imputed 1 numeric column(s) using mean.
Missing data check complete. No columns have missing data.


In [4]:
# bin rare categorical feature values. If the column is full of rare categorical values, then drop it.
df = myFn.bin_groups(df,categoricalNumericFeatures, percent=.05)
df = myFn.drop_cols_too_unique(df,percent=.8)
df.head()

dropped Name, 100.0% unique
dropped Ticket, 100.0% unique


,Sex,Embarked,Pclass,Survived,Age,SibSp,Parch,Fare
PassengerId,,,,,,,,
1,male,S,3,0,22.0,1.0,0.0,7.2500
2,female,C,1,1,38.0,1.0,0.0,71.2833
3,female,S,3,1,26.0,0.0,0.0,7.9250
4,female,S,1,1,35.0,1.0,0.0,53.1000
5,male,S,3,0,35.0,0.0,0.0,8.0500


In [5]:
# dummy code
label = 'Survived'
df = myFn.get_dummies(df,categoricalNumericFeatures, label = label)

In [6]:
from sklearn import preprocessing

dfMinMax = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df),columns=df.columns)
#dfMinMax.hist(figsize=(15,10))

In [7]:
X = dfMinMax.drop(columns=label)
y = dfMinMax[label]

#### Here is the estimated percent of the test set that will be classified correctly. In this case I don't really use it for anything since I just use logistic regression anyway.

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
from numpy import mean


cv = KFold(n_splits=10, random_state=12345, shuffle=True)

model_log = LogisticRegression(max_iter=100)
pctCorrect =  mean(cross_val_score(model_log, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
print(pctCorrect)

0.8024843945068664


In [9]:
model_log.fit(X,y)

LogisticRegression()

In [10]:
categoricalNumericFeatures = ['Pclass'] # pclass is numeric but actually is a categorical feature
missingData = MissingData(df_test, categoricalImputer='most_frequent',numericImputer='mean', categoricalNumericFeatures=categoricalNumericFeatures, dropColPct=.7)
missingData.missing_values_report()

Pclass	0.0%
Name	0.0%
Sex	0.0%
Age	20.57%
SibSp	0.0%
Parch	0.0%
Ticket	0.0%
Fare	0.24%
Cabin	78.23%
Embarked	0.0%


In [11]:
df_test = missingData.fillMissingData()

dropped the Cabin column. Over 70% of records were Null.
imputed 1 categorical column(s) using most_frequent.
imputed 2 numeric column(s) using mean.
Missing data check complete. No columns have missing data.


In [12]:
# bin rare categorical feature values. If the column is full of rare categorical values, then drop it.
df_test = myFn.bin_groups(df_test,categoricalNumericFeatures, percent=.05)
df_test = myFn.drop_cols_too_unique(df_test,percent=.8)
df_test = myFn.get_dummies(df_test,categoricalNumericFeatures)

dropped Name, 100.0% unique
dropped Ticket, 100.0% unique


In [13]:
dfMinMax2 = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df_test),columns=df_test.columns, index=df_test.index)

In [14]:
prediction = model_log.predict(dfMinMax2)
predictions = {'PassengerId':dfMinMax2.index.array,'Survived':prediction}
dfPredict = pd.DataFrame(predictions,dtype=int)

In [15]:
dfPredict.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [16]:
dfPredict.to_csv('predictions.csv',index=False)